In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

def fourier_squared_distance(list_of_coeffs, param_list):
    """
    Returns the squared l2-distance in Fourier space between a function
    characterized by its Fourier coefficients and the output of the
    quantum model

    Args:
        list_of coeffs (list(float)): A list of seven coefficients
                                      corresponding to the Fourier
                                      coefficients of the function we
                                      want to approximate
        param_list (list(float)): A list of six parameters characterizing
                                  the angles in the trainable circuit.

    Returns: (float): Squared l2-distance between the given function
                      and the output of the quantum model
    """

    dev = qml.device("default.qubit", wires=3)

    # Feel free to define any helper functions, such as subcircuits, here.
    def trainalbe_circuit(x):
    # trainable circuit
        for i in range(len(x)):
            qml.RX(x[i],wires=[i])
        n,m = 3,2
        circular = [tuple(range(n - m + 1, n)) + (0,)] + [tuple(range(i, i + m)) for i in range(n - m + 1)]
        for j in sorted(circular):
            qml.CNOT(wires=[j[0],j[1]])

    @qml.qnode(dev)
    def circuit(x,param_list):
        """This circuit returns the PauliZ expectation of
        the quantum model in the statement"""
        # Put your code here 
        trainalbe_circuit(param_list[:3]) # qml.BasicEntanglerLayers()
        #encoding
        for j in range(3):
            qml.RX(x,wires=[j], id="x")
        trainalbe_circuit(param_list[3:])
        return qml.expval(qml.PauliZ(0)) # qml.BasicEntanglerLayers()

    # Write a function that calculates the squared l2-distance here
    circ = functools.partial(circuit,param_list=param_list)
    coeffs = qml.fourier.coefficients(circ, 1, 3)
    l2_list = []
    for w in range(len(list_of_coeffs)):
        l2_list.append(np.abs(list_of_coeffs[w]*np.exp(1j*list_of_coeffs[w])-coeffs[w]*(np.exp(1j*list_of_coeffs[w])))**2)

    # Return your final answer here
    return np.sum(l2_list)

In [2]:
fourier_squared_distance([-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5])
    #[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9])

0.003676608593303433

In [3]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = fourier_squared_distance(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated squared distance isn't quite right."


test_cases = [['[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]', '0.0036766085933034303'], ['[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]', '0.6538589174369286']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]'...
Correct!
Running test case 1 with input '[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]'...
Correct!
